In [13]:

import pandas as pd
import sqlite3
from datetime import datetime

def get_date_df(date):
    date_str = date.strftime("%Y-%m-%d")
    con = sqlite3.connect('../data/db.sqlite3')
    date_df = pd.read_sql_query('SELECT * from sample_tb WHERE \''+ date_str + ' 00:00\' < ttn_received_at AND ttn_received_at < \'' + date_str + ' 23:59\'', con)
    con.close()
    return date_df

def get_today_df():
    date = datetime.now()
    date_str = date.strftime("%Y-%m-%d")
    con = sqlite3.connect('../data/db.sqlite3')
    today_df = pd.read_sql_query('SELECT * from sample_tb WHERE \''+ date_str + ' 00:00\' < ttn_received_at AND ttn_received_at < \'' + date_str + ' 23:59\'', con)
    con.close()
    return today_df

date = datetime(2023, 1, 23)
date_df = get_date_df(date)

date_df = date_df[['ttn_device_id', 'ttn_received_at', 'ttn_payload_temp', 'ttn_payload_rh', 'ttn_payload_pm1_0', 'ttn_payload_pm2_5', 'ttn_payload_pm10_0']]
date_df

,ttn_device_id,ttn_received_at,ttn_payload_temp,ttn_payload_rh,ttn_payload_pm1_0,ttn_payload_pm2_5,ttn_payload_pm10_0
0,eui-70b3d57ed0059066,2023-01-23 01:49:01.717729,31.70,60.90,2.0,3.0,3.0
1,eui-70b3d57ed0059066,2023-01-23 02:04:01.758256,31.00,63.79,2.0,4.0,4.0
2,eui-70b3d57ed0059066,2023-01-23 02:19:01.786385,31.29,62.40,0.0,1.0,2.0
3,eui-70b3d57ed0059066,2023-01-23 01:49:01.717729,31.70,60.90,2.0,3.0,3.0
4,eui-70b3d57ed0059066,2023-01-23 02:04:01.758256,31.00,63.79,2.0,4.0,4.0
...,...,...,...,...,...,...,...
87,eui-70b3d57ed0059066,2023-01-23 22:48:50.088917,29.29,64.19,5.0,6.0,6.0
88,eui-70b3d57ed0059066,2023-01-23 23:03:50.122912,29.29,64.09,4.0,6.0,6.0
89,eui-70b3d57ed0059066,2023-01-23 23:18:50.146435,29.29,64.00,4.0,6.0,8.0
90,eui-70b3d57ed0059066,2023-01-23 23:33:50.197491,29.29,64.09,4.0,6.0,9.0


In [14]:

def apply_func(ttn_received_at):
	date = datetime.fromisoformat(str(ttn_received_at))	
	return date.replace(microsecond=0).replace(second=0).replace(minute=0).replace(hour=0)
date_df['ttn_received_at'] = date_df.apply(lambda x: apply_func(x['ttn_received_at']), axis=1)

date_df = date_df.groupby(['ttn_device_id', 'ttn_received_at']).agg(
    ttn_payload_temp = ('ttn_payload_temp', 'mean'),
    ttn_payload_rh = ('ttn_payload_rh', 'mean'),
    ttn_payload_pm1_0 = ('ttn_payload_pm1_0', 'mean'),
    ttn_payload_pm2_5 = ('ttn_payload_pm2_5', 'mean'),
    ttn_payload_pm10_0 = ('ttn_payload_pm10_0', 'mean')
).reset_index()

date_df= date_df.round(decimals=2)

date_df


,ttn_device_id,ttn_received_at,ttn_payload_temp,ttn_payload_rh,ttn_payload_pm1_0,ttn_payload_pm2_5,ttn_payload_pm10_0
0,eui-70b3d57ed0059066,2023-01-23,28.65,67.56,4.21,6.63,7.26


In [18]:

def apply_func(ttn_received_at):
	return ttn_received_at.strftime('%Y-%m-%d')
date_df['ttn_received_at'] = date_df.apply(lambda x: apply_func(x['ttn_received_at']), axis=1)
date_df


,ttn_device_id,ttn_received_at,ttn_payload_temp,ttn_payload_rh,ttn_payload_pm1_0,ttn_payload_pm2_5,ttn_payload_pm10_0
0,eui-70b3d57ed0059066,2023-01-23,28.65,67.56,4.21,6.63,7.26


In [16]:
date_dicts = date_df.to_dict('records')
date_df.to_dict('records')

[{'ttn_device_id': 'eui-70b3d57ed0059066',
  'ttn_received_at': Timestamp('2023-01-23 00:00:00'),
  'ttn_payload_temp': 28.65,
  'ttn_payload_rh': 67.56,
  'ttn_payload_pm1_0': 4.21,
  'ttn_payload_pm2_5': 6.63,
  'ttn_payload_pm10_0': 7.26}]

In [3]:

import pandas as pd
import sqlite3
from datetime import datetime

def get_select_df():
    date_start_str = datetime.now().strftime("%Y-%m-%d")
    con = sqlite3.connect('../data/db.sqlite3')
    select_df = pd.read_sql_query('SELECT * from sample_tb WHERE \''+ date_start_str + ' 00:00\' < ttn_received_at', con)
    con.close()    
    return select_df

get_select_df()

,sensorar_sample_id,ttn_gateway_id,ttn_gateway_lat,ttn_gateway_lng,ttn_device_id,ttn_received_at,ttn_payload_frm,ttn_payload_temp,ttn_payload_rh,ttn_payload_pm1_0,ttn_payload_pm2_5,ttn_payload_pm10_0


In [6]:
import json

uplink_dicts = json.load('../data/cached_last_uplink_messages.json')

items = []
for uplink_dict in uplink_dicts:
    item = {
        'ttn_gateway_id': uplink_dict['uplink_message']['rx_metadata'][0]['gateway_ids']['gateway_id'],
        'ttn_gateway_lat': uplink_dict['uplink_message']['rx_metadata'][0]['location']['latitude'],
        'ttn_gateway_lng': uplink_dict['uplink_message']['rx_metadata'][0]['location']['longitude'],
        'ttn_device_id': uplink_dict['end_device_ids']['device_id'],
        'ttn_received_at': datetime.fromisoformat(str(uplink_dict['received_at'])),
        'ttn_payload_frm': uplink_dict['uplink_message']['frm_payload'],
        'ttn_payload_temp': uplink_dict['uplink_message']['decoded_payload']['temp'],
        'ttn_payload_rh': uplink_dict['uplink_message']['decoded_payload']['rh'],
        'ttn_payload_pm1_0': uplink_dict['uplink_message']['decoded_payload']['pm1_0'],
        'ttn_payload_pm2_5': uplink_dict['uplink_message']['decoded_payload']['pm2_5'],
        'ttn_payload_pm10_0': uplink_dict['uplink_message']['decoded_payload']['pm10_0']      
    }
    items.append(item)

print(items)

AttributeError: 'str' object has no attribute 'read'

In [17]:
import json

with open('../data/cached_last_uplink_messages.json') as json_file:    
    uplink_dicts = json.load(json_file)
    items = []
    for uplink_dict in uplink_dicts:
        item = {
            'ttn_gateway_id': uplink_dict['result']['uplink_message']['rx_metadata'][0]['gateway_ids']['gateway_id'],
            'ttn_gateway_lat': uplink_dict['result']['uplink_message']['rx_metadata'][0]['location']['latitude'],
            'ttn_gateway_lng': uplink_dict['result']['uplink_message']['rx_metadata'][0]['location']['longitude'],
            'ttn_device_id': uplink_dict['result']['end_device_ids']['device_id'],
            'ttn_received_at': datetime.fromisoformat(str(uplink_dict['result']['received_at'])),
            'ttn_payload_frm': uplink_dict['result']['uplink_message']['frm_payload'],
            'ttn_payload_temp': uplink_dict['result']['uplink_message']['decoded_payload']['temp'],
            'ttn_payload_rh': uplink_dict['result']['uplink_message']['decoded_payload']['rh'],
            'ttn_payload_pm1_0': uplink_dict['result']['uplink_message']['decoded_payload']['pm1_0'],
            'ttn_payload_pm2_5': uplink_dict['result']['uplink_message']['decoded_payload']['pm2_5'],
            'ttn_payload_pm10_0': uplink_dict['result']['uplink_message']['decoded_payload']['pm10_0']      
        }
        items.append(item)

    print(items)

[{'ttn_gateway_id': 'eui-a840412115204150', 'ttn_gateway_lat': -9.376738741493659, 'ttn_gateway_lng': -40.490646447929585, 'ttn_device_id': 'eui-70b3d57ed0059066', 'ttn_received_at': datetime.datetime(2023, 2, 1, 12, 11, 55, 188648, tzinfo=datetime.timezone.utc), 'ttn_payload_frm': 'ABwAMgA9AB0AAAAAAAA=', 'ttn_payload_temp': 28.5, 'ttn_payload_rh': 61.29, 'ttn_payload_pm1_0': 0, 'ttn_payload_pm2_5': 0, 'ttn_payload_pm10_0': 0}, {'ttn_gateway_id': 'eui-a840412115204150', 'ttn_gateway_lat': -9.376738741493659, 'ttn_gateway_lng': -40.490646447929585, 'ttn_device_id': 'eui-70b3d57ed0059066', 'ttn_received_at': datetime.datetime(2023, 2, 1, 12, 26, 55, 216430, tzinfo=datetime.timezone.utc), 'ttn_payload_frm': 'ABwARgA9AAAAAQABAAE=', 'ttn_payload_temp': 28.7, 'ttn_payload_rh': 61, 'ttn_payload_pm1_0': 1, 'ttn_payload_pm2_5': 1, 'ttn_payload_pm10_0': 1}, {'ttn_gateway_id': 'eui-a840412115204150', 'ttn_gateway_lat': -9.376738741493659, 'ttn_gateway_lng': -40.490646447929585, 'ttn_device_id': '

In [37]:
import requests
import json

def _request_uplink_messages_response():
    headers = {
        'Authorization': f'Bearer NNSXS.2X5P5567GPCXENQYCPINZ5BAPA62GEC73R5NSVY.DJ7PAPBZDG6WV3FDSDMMRD4L3LPSDQBJNFK2BLLXTF6PXZXHKJVA',
        'Accept': 'text/event-stream'
    }
    params ={
        'after': '2023-01-22T00:00:00Z',
        'field_mask': 'up.uplink_message'
    }
    response = requests.get(
        (
            f'https://nam1.cloud.thethings.network/api/v3/as'
            f'/applications/sensorar-application-1/packages/storage/uplink_message'
        ),
        headers=headers,
        params=params
    )
    return response

def _assemply_response_uplink_messages(uplink_messages_response):
    
    content_str = '[' + uplink_messages_response.content.decode("utf-8") + ']'
    content_str = content_str.replace('\n', '')
    content_str = content_str.replace('{"result":', ',{"result":')
    content_str = content_str.replace('[,{"result":', '[{"result":')

    response_uplink_messages = json.loads(content_str)
    return response_uplink_messages    

uplink_messages_response = _request_uplink_messages_response()
response_uplink_dicts = _assemply_response_uplink_messages(uplink_messages_response)

uplink_dicts = []
for response_uplink_dict in response_uplink_dicts:
    uplink_dict = {
        'ttn_gateway_id': response_uplink_dict['result']['uplink_message']['rx_metadata'][0]['gateway_ids']['gateway_id'],
        'ttn_gateway_lat': response_uplink_dict['result']['uplink_message']['rx_metadata'][0]['location']['latitude'],
        'ttn_gateway_lng': response_uplink_dict['result']['uplink_message']['rx_metadata'][0]['location']['longitude'],
        'ttn_device_id': response_uplink_dict['result']['end_device_ids']['device_id'],
        'ttn_received_at': datetime.fromisoformat(str(response_uplink_dict['result']['received_at'])),
        'ttn_payload_frm': response_uplink_dict['result']['uplink_message']['frm_payload'],
        'ttn_payload_temp': response_uplink_dict['result']['uplink_message']['decoded_payload']['temp'],
        'ttn_payload_rh': response_uplink_dict['result']['uplink_message']['decoded_payload']['rh'],
        'ttn_payload_pm1_0': response_uplink_dict['result']['uplink_message']['decoded_payload']['pm1_0'],
        'ttn_payload_pm2_5': response_uplink_dict['result']['uplink_message']['decoded_payload']['pm2_5'],
        'ttn_payload_pm10_0': response_uplink_dict['result']['uplink_message']['decoded_payload']['pm10_0']      
    }
    uplink_dicts.append(uplink_dict)

uplink_df = pd.DataFrame.from_dict(uplink_dicts)
uplink_df = uplink_df.astype(str)
uplink_df


,ttn_gateway_id,ttn_gateway_lat,ttn_gateway_lng,ttn_device_id,ttn_received_at,ttn_payload_frm,ttn_payload_temp,ttn_payload_rh,ttn_payload_pm1_0,ttn_payload_pm2_5,ttn_payload_pm10_0
0,eui-a840412115204150,-9.376738741493659,-40.490646447929585,eui-70b3d57ed0059066,2023-02-22 00:41:43.993145+00:00,AB8AFAAvABQAAAAAAAA=,31.2,47.2,0,0,0
1,eui-a840412115204150,-9.376738741493659,-40.490646447929585,eui-70b3d57ed0059066,2023-02-22 00:56:42.515405+00:00,AB8AHQAuAFoAAgADAAU=,31.29,46.9,2,3,5
2,eui-a840412115204150,-9.376738741493659,-40.490646447929585,eui-70b3d57ed0059066,2023-02-22 01:11:42.548451+00:00,ACAAAAAtABQAAAAAAAA=,32.0,45.2,0,0,0
3,eui-a840412115204150,-9.376738741493659,-40.490646447929585,eui-70b3d57ed0059066,2023-02-22 01:26:42.581140+00:00,ACAAFAAtAAAAAQACAAQ=,32.2,45.0,1,2,4


In [53]:


def get_last_timestamp_str():
    con = sqlite3.connect('../data/db.sqlite3')
    select_df = pd.read_sql_query('SELECT MAX(ttn_received_at) as ttn_received_at FROM sample_tb', con)
    con.close()
    if(select_df.loc[0, 'ttn_received_at'] is not None):  
        last_timestamp_str = str(select_df.loc[0, 'ttn_received_at'])    
        return last_timestamp_str
    return '2000-01-1 00:00:00.000000+00:00'

last_timestamp_str = get_last_timestamp_str()

inserts_df = uplink_df[uplink_df['ttn_received_at'] > last_timestamp_str].reset_index(drop=True)
inserts_df = inserts_df.astype(str)
inserts_df

last_timestamp_str

None


'None'